In [ ]:
# https://github.com/nilmtk/nilmtk/blob/master/docs/manual/user_guide/nilmtk_api_tutorial.ipynb

# NILMTK Rapid Experimentation API

### This notebook demonstrates the use of NILMTK's ExperimentAPI - a  new  NILMTK  interface which  allows NILMTK users to focus on which experiments to run rather than on the code required to run such experiments.

It is important to note that handing over so much flexibility to the user does require the user to be somewhat familiar with the data set, but this part of the process is supported by NILMTK as data exploration is simple and well documented. 

Lets us start with a very simple experiment to demonstrate the use of the API for multiple appliances in a minimal use case. This experiment shows how the user can select the appliances in the dataset on which disaggregation is to be performed.

Importing the API. 

In [1]:
from nilmtk.api import API
import warnings
warnings.filterwarnings("ignore")

Next, we import the required algorithms on which we wish to run the experiments

In [2]:
from nilmtk.disaggregate import CO

Next, we enter the values for the different parameters in the dictionary. Since we need multiple appliances, we enter the names of all the required appliances in the _'appliances'_ parameter.

In [3]:
experiment1 = {
  'power': {'mains': ['apparent','active'],'appliance': ['apparent','active']},
  'sample_rate': 60,
  # 'appliances': ['fridge','air conditioner', 'microwave'],
  'appliances': ['fridge','dish washer', 'microwave'],
  'methods': {"CO":CO({})},
  'train': {    
    'datasets': {
        #'Dataport': {
        #    'path': 'data/dataport.hdf5',
        #    'buildings': {
        #        10: {
        #            'start_time': '2015-04-04',
        #            'end_time': '2015-04-06'
        #            }
        #        }                
        #    }
        'redd': {
            'path': '../../dataset/redd/redd.h5',
            'buildings': {
                1: {
                    'start_time': '2011-04-26',
                    'end_time': '2011-04-29'
                    }
                }                
            }
        }
    },
  'test': {
    'datasets': {
        #'Dataport': {
        #    'path': 'data/dataport.hdf5',
        #    'buildings': {
        #        10: {
        #            'start_time': '2015-04-25',
        #            'end_time': '2015-04-26'
        #            }
        #        }
        #    }
        'redd': {
            'path': '../../dataset/redd/redd.h5',
            'buildings': {
                1: {
                    'start_time': '2011-04-30',
                    'end_time': '2011-05-01'
                    }
                }
            }
        },
        'metrics':['rmse']
    }
}

In this example experimental setup, we have set the _sample rate_ at 60Hz and use Combinatorial Optimisation to 
disaggregate the required appliances from building 10 in the dataport dataset with the _RMSE_ metric to measure the accuracy. We also specify the dates for training and testing

Next we provide this experiment dictionary as input to the API.

In [4]:
api_results_experiment_1 = API(experiment1)

Joint Testing for all algorithms
Loading data for  redd  dataset
Loading data for meter ElecMeterID(instance=2, building=1, dataset='REDD')     
Done loading data all meters for this chunk.
Dropping missing values
Generating predictions for : CO
...............CO disaggregate_chunk running.............
............  rmse  ..............wave'r'
                     CO
fridge       199.626859
dish washer  446.380163
microwave    519.516301


We can observe the prediction vs. truth graphs in the above cell. The accuracy metrics can be accessed using the following commands:

In [5]:
errors_keys = api_results_experiment_1.errors_keys
errors = api_results_experiment_1.errors
for i in range(len(errors)):
    print (errors_keys[i])
    print (errors[i])
    print ("\n\n")

redd_1_rmse
                     CO
fridge       199.626859
dish washer  446.380163
microwave    519.516301





This was a trivial experiment that only scratches the surface of the true potential of this API.

In the next experiment we will run an incrementally more complex version of the above experiment. Here we will use multiple models to disaggregate the appliance readings with the models having their own sets of parameters which can be set by the users within the experimental dictionary in order to fine tune experiments.

We also import the required algorithms for the next experiments

In [6]:
# from nilmtk.disaggregate import FHMM_EXACT, Mean
from nilmtk.disaggregate import FHMMExact, Mean

In [7]:
experiment2 = {
  'power': {'mains': ['apparent','active'],'appliance': ['apparent','active']},
  'sample_rate': 60,
  # 'appliances': ['fridge','air conditioner', 'microwave'],
  'appliances': ['fridge','dish washer', 'microwave'],
  # 'methods': {"Mean":Mean({}),"FHMM_EXACT":FHMM_EXACT({'num_of_states':2}), "CombinatorialOptimisation":CO({})},
  'methods': {"Mean":Mean({}),"FHMMExact":FHMMExact({'num_of_states':2}), "CombinatorialOptimisation":CO({})},
  'train': {    
    'datasets': {
        #'Dataport': {
        #    'path': 'data/dataport.hdf5',
        #    'buildings': {
        #        10: {
        #            'start_time': '2015-04-04',
        #            'end_time': '2015-04-06'
        #            }
        #        }                
        #    }
        'redd': {
            'path': '../../dataset/redd/redd.h5',
            'buildings': {
                1: {
                    'start_time': '2011-04-26',
                    'end_time': '2011-04-29'
                    }
                }                
            }
        }
    },
  'test': {
    'datasets': {
        #'Dataport': {
        #    'path': 'data/dataport.hdf5',
        #    'buildings': {
        #        10: {
        #            'start_time': '2015-04-25',
        #            'end_time': '2015-04-26'
        #            }
        #        }
        #    }
        'redd': {
            'path': '../../dataset/redd/redd.h5',
            'buildings': {
                1: {
                    'start_time': '2011-04-30',
                    'end_time': '2011-05-01'
                    }
                }
            }
        },
        'metrics':['mae', 'rmse']
    }
}

In [8]:
api_results_experiment_2 = API(experiment2)

Joint Testing for all algorithms
Loading data for  redd  dataset
Loading data for meter ElecMeterID(instance=2, building=1, dataset='REDD')     
Done loading data all meters for this chunk.
Dropping missing values
Generating predictions for : Mean
Generating predictions for : FHMM
Generating predictions for : CO
...............CO disaggregate_chunk running.............
............  mae  ..............owave'r'
                  Mean   FHMMExact  CombinatorialOptimisation
fridge       69.150978   83.206024                 163.684143
dish washer  78.050056  167.286194                 241.650696
microwave    34.286942   81.578827                 275.645447
............  rmse  ..............
                   Mean   FHMMExact  CombinatorialOptimisation
fridge        80.203093  121.692612                 199.626859
dish washer  253.148934  255.244070                 446.380163
microwave    161.838739  171.196627                 519.516301


In [9]:
api_results_experiment_2.errors

[                  Mean   FHMMExact  CombinatorialOptimisation
 fridge       69.150978   83.206024                 163.684143
 dish washer  78.050056  167.286194                 241.650696
 microwave    34.286942   81.578827                 275.645447,
                    Mean   FHMMExact  CombinatorialOptimisation
 fridge        80.203093  121.692612                 199.626859
 dish washer  253.148934  255.244070                 446.380163
 microwave    161.838739  171.196627                 519.516301]

In [10]:
errors_keys = api_results_experiment_2.errors_keys
errors = api_results_experiment_2.errors
for i in range(len(errors)):
    print (errors_keys[i])
    print (errors[i])
    print ("\n\n")

redd_1_mae
                  Mean   FHMMExact  CombinatorialOptimisation
fridge       69.150978   83.206024                 163.684143
dish washer  78.050056  167.286194                 241.650696
microwave    34.286942   81.578827                 275.645447



redd_1_rmse
                   Mean   FHMMExact  CombinatorialOptimisation
fridge        80.203093  121.692612                 199.626859
dish washer  253.148934  255.244070                 446.380163
microwave    161.838739  171.196627                 519.516301





The API makes running experiments extremely quick and efficient, with the emphasis on creating finely tuned reproducible experiments where model and parameter performances can be easily evaluated at a glance.   

In the next iteration of this experiment, we introduce more parameters _chunksize_, _DROP_ALL_NANS_ and _artificial_aggregate_ and add another disaggregation algorithm (_Hart85_). We also train and test data from multiple buildings of the same dataset.

We also import the Hart algorithm for the next experiment

In [11]:
from nilmtk.disaggregate import Hart85

In [12]:
experiment3 = {
  'power': {'mains': ['apparent','active'],'appliance': ['apparent','active']},
  'sample_rate': 60,
  # 'appliances': ['fridge','air conditioner','electric furnace','washing machine'],
  'appliances': ['fridge','dish washer', 'microwave'],
  'artificial_aggregate': True,
  'chunksize': 20000,
  'DROP_ALL_NANS': False,
  # 'methods': {"Mean":Mean({}),"Hart85":Hart85({}), "FHMM_EXACT":FHMM_EXACT({'num_of_states':2}), "CO":CO({})},
  'methods': {"Mean":Mean({}),"Hart85":Hart85({}), "FHMMExact":FHMMExact({'num_of_states':2}), "CO":CO({})},
  'train': {    
    'datasets': {
      #'Dataport': {
      #  'path': 'data/dataport.hdf5',
      #  'buildings': {
      #    54: {
      #      'start_time': '2015-01-28',
      #      'end_time': '2015-02-12'
      #    },
      #    56: {
      #      'start_time': '2015-01-28',
      #      'end_time': '2015-02-12'
      #    },
      #    57: {
      #      'start_time': '2015-04-30',
      #      'end_time': '2015-05-14'
      #    },
      #          }                
      #      }
        'redd': {
            'path': '../../dataset/redd/redd.h5',
            'buildings': {
                1: {
                    'start_time': '2011-04-20',
                    'end_time': '2011-04-21'
                    }
                }                
            }
        }
    },
  'test': {
    'datasets': {
      #'Datport': {
      #  'path': 'data/dataport.hdf5',
      #  'buildings': {
      #    94: {
      #      'start_time': '2015-04-30',
      #      'end_time': '2015-05-07'
      #    },
      #    103: {
      #      'start_time': '2014-01-26',
      #      'end_time': '2014-02-03'
      #    },
      #    113: {
      #      'start_time': '2015-04-30',
      #      'end_time': '2015-05-07'
      #    },
      #  }
      #}
         'redd': {
            'path': '../../dataset/redd/redd.h5',
            'buildings': {
                1: {
                    'start_time': '2011-04-30',
                    'end_time': '2011-05-01'
                    },
                2: {
                    'start_time': '2011-04-30',
                    'end_time': '2011-05-01'
                    },
                }
            }        
    },
        'metrics':['mae', 'rmse']
    }
}

In [13]:
api_results_experiment_3 = API(experiment3)

Joint Testing for all algorithms
Loading data for  redd  dataset
Loading data for meter ElecMeterID(instance=2, building=1, dataset='REDD')     
Done loading data all meters for this chunk.
Creating an Artificial Aggregate
Generating predictions for : Mean
Generating predictions for : Hart85
Finding Edges, please wait ...
Edge detection complete.
Creating transition frame ...
Transition frame created.
Creating states frame ...
States frame created.
Finished.
Generating predictions for : FHMM
Generating predictions for : CO
...............CO disaggregate_chunk running.............
............  mae  ..............owave'r'
                  Mean     Hart85  FHMMExact         CO
fridge       69.544853  14.712614  69.594574  20.867754
dish washer  68.440933  99.631096  68.422577  68.422577
microwave    41.142899  89.011604  54.325802  77.547081
............  rmse  ..............
                   Mean      Hart85   FHMMExact          CO
fridge        80.210876   38.493287   80.212410   50

In [14]:
errors_keys = api_results_experiment_3.errors_keys
errors = api_results_experiment_3.errors
for i in range(len(errors)):
    print (errors_keys[i])
    print (errors[i])
    print ("\n\n")

redd_1_mae
                  Mean     Hart85  FHMMExact         CO
fridge       69.544853  14.712614  69.594574  20.867754
dish washer  68.440933  99.631096  68.422577  68.422577
microwave    41.142899  89.011604  54.325802  77.547081



redd_1_rmse
                   Mean      Hart85   FHMMExact          CO
fridge        80.210876   38.493287   80.212410   50.079000
dish washer  256.203319  248.144513  256.218154  256.218154
microwave    161.565752  344.156294  182.498266  288.965382



redd_2_mae
                  Mean     Hart85  FHMMExact         CO
fridge       82.871750  32.943031  82.867722  32.407295
dish washer  23.114321  92.584518  23.095264  23.095264
microwave    35.856228  44.888508  39.568760  44.986118



redd_2_rmse
                   Mean      Hart85   FHMMExact          CO
fridge        95.468909   61.741750   95.429616   60.662867
dish washer  152.608011  172.967895  152.616407  152.616407
microwave    152.074226  217.367111  154.138793  179.959989





The results of the above experiment are presented for every chunk per building in the test set.

In the following experiment, we demonstrate how to run experiments across datasets, which was previously not possible. The important thing to pay attention to is that such datasets can only be trained and tested together as long as they have common appliances in homes with common _ac_types_.

In [15]:
experiment4 = {
  'power': {'mains': ['apparent','active'],'appliance': ['apparent','active']},
  'sample_rate': 60,
  'appliances': ['washing machine','fridge'],
  'artificial_aggregate': True,
  'chunksize': 20000,
  'DROP_ALL_NANS': False,
  # 'methods': {"Mean":Mean({}),"Hart85":Hart85({}), "FHMM_EXACT":FHMM_EXACT({'num_of_states':2}), 'CO':CO({})},   
  'methods': {"Mean":Mean({}),"Hart85":Hart85({}), "FHMMExact":FHMMExact({'num_of_states':2}), 'CO':CO({})}, 
  'train': {
    'datasets': {
      'UKDALE': {
        # 'path': 'C:/Users/Hp/Desktop/nilmtk-contrib/ukdale.h5',
        'path': '../../dataset/ukdale/ukdale.h5',
        'buildings': {
              1: {
                'start_time': '2017-01-05',
                # 'end_time': '2017-03-05'
                'end_time': '2017-01-06'
              },          
            }
          },        
        }
      },    
  'test': {
    'datasets': {
      #'DRED': {
      #  'path': 'C:/Users/Hp/Desktop/nilmtk-contrib/dred.h5',
      #  'buildings': {
      #        1: {
      #              'start_time': '2015-09-21',
      #              'end_time': '2015-10-01'
      #    }
      #  }
      #},
      'REDD': {
        # 'path': 'C:/Users/Hp/Desktop/nilmtk-contrib/redd.h5',
        'path': '../../dataset/redd/redd.h5',
        'buildings': {
              1: {
                    # 'start_time': '2011-04-17',
                    # 'end_time': '2011-04-27'
                    'start_time': '2011-04-30',
                    'end_time': '2011-05-01'
          }
        }
      }
    },
        'metrics':['mae', 'rmse']
    }
}

In [17]:
api_results_experiment_4 = API(experiment4)

Joint Testing for all algorithms
Loading data for  REDD  dataset
Loading data for meter ElecMeterID(instance=2, building=1, dataset='REDD')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=20, building=1, dataset='REDD')     
Done loading data all meters for this chunk.
Creating an Artificial Aggregate
Generating predictions for : Mean
Generating predictions for : Hart85
Finding Edges, please wait ...
Edge detection complete.
Creating transition frame ...
Transition frame created.
Creating states frame ...
States frame created.
Finished.
Generating predictions for : FHMM
Generating predictions for : CO
...............CO disaggregate_chunk running.............
............  mae  ..............ge' machine'
                       Mean      Hart85   FHMMExact         CO
washing machine  376.332214  161.648666  143.177536  85.860405
fridge            60.663910   38.429340   37.200783  47.971703
............  rmse  ..............
                 

In [18]:
errors_keys = api_results_experiment_4.errors_keys
errors = api_results_experiment_4.errors
for i in range(len(errors)):
    print (errors_keys[i])
    print (errors[i])
    print ("\n\n")

REDD_1_mae
                       Mean      Hart85   FHMMExact         CO
washing machine  376.332214  161.648666  143.177536  85.860405
fridge            60.663910   38.429340   37.200783  47.971703



REDD_1_rmse
                       Mean      Hart85   FHMMExact          CO
washing machine  672.054034  566.647736  496.224102  218.785881
fridge            81.935459   58.822408   59.372118   84.568095





Just like the above experiments, any user can set up other experiments very quickly. 